In [6]:
import os
import pandas as pd
import numpy as np
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
%matplotlib inline
sns.set(color_codes=True)

In [7]:
# TODO: change this to yours ...
DATA_DIR = "/content/"

# read 3 csv files and 1 pkl file
train_csv_file = os.path.join(DATA_DIR, "train.csv")
val_csv_file = os.path.join(DATA_DIR, "valid.csv")
test_csv_file = os.path.join(DATA_DIR, "test.csv")
ehr_pkl_file = os.path.join(DATA_DIR, "ehr_preprocessed_seq_by_day_cat_embedding.pkl")

train_df = pd.read_csv(train_csv_file)
val_df = pd.read_csv(val_csv_file)
test_df = pd.read_csv(test_csv_file)

with open(ehr_pkl_file, 'rb') as f:
    ehr_data = pd.read_pickle(f)

In [5]:
"""
#Option1 : Aggregate features by mean over the days

# Extract EHR features and IDs
ehr_features = ehr_data["feat_dict"]
ehr_ids = list(ehr_features.keys())

# Convert to DataFrame (each row is an admission)
ehr_df = pd.DataFrame({
    "id": ehr_ids,
    "ehr_matrix": [ehr_features[id] for id in ehr_ids]
})

# Merge with labels from train/val/test DataFrames
def merge_labels(df, ehr_df):
    return df[["id", "readmitted_within_30days"]].merge(ehr_df, on="id", how="inner")

train_ehr = merge_labels(train_df, ehr_df)
val_ehr = merge_labels(val_df, ehr_df)


def aggregate_ehr(df):
    # Aggregate features (example: mean over days)
    df["ehr_mean"] = df["ehr_matrix"].apply(lambda x: np.mean(x, axis=0))

    # Convert to numpy arrays
    X = np.stack(df["ehr_mean"].values)
    y = df["readmitted_within_30days"].astype(int).values

    return X, y

X_train, y_train = aggregate_ehr(train_ehr)
X_val, y_val = aggregate_ehr(val_ehr)
"""

In [9]:
X_train = []
for idx, row in train_df.iterrows():
    X_train.append(ehr_data["feat_dict"][row["id"]][-1])
X_train = np.array(X_train)
print(X_train.shape)
print('train_data:',X_train,'\n')

X_val = []
for idx, row in val_df.iterrows():
    X_val.append(ehr_data["feat_dict"][row["id"]][-1])
X_val = np.array(X_val)
print(X_val.shape)
print('validation_data:,',X_val,'\n')

y_train = train_df['readmitted_within_30days'].values
y_val = val_df['readmitted_within_30days'].values

(49451, 171)
train_data: [[52  1  6 ...  0  0  0]
 [52  1  6 ...  0  0  0]
 [52  1  6 ...  0  0  0]
 ...
 [91  1  6 ...  0  0  1]
 [69  0  6 ...  0  0  0]
 [69  0  6 ...  0  0  0]] 

(16721, 171)
validation_data:, [[75  0  6 ...  3  6  0]
 [75  0  6 ...  3  6  0]
 [75  0  6 ...  3  6  0]
 ...
 [77  1  6 ...  0  0  0]
 [49  1  6 ...  0  0  0]
 [49  1  6 ...  0  0  0]] 



In [8]:
from scipy.stats import pointbiserialr
# initialize storage for std_devs
std_devs = np.std(X_train, axis=0)


feature_names = ehr_data["feature_cols"]

# Create a DataFrame for analysis
std_df = pd.DataFrame({
    "Feature": feature_names,
    "Std_Dev": std_devs
})

# Initialize storage
correlations = []
p_values = []

# Calculate correlation for each feature
for i in range(X_train.shape[1]):
    if np.std(X_train[:, i]) == 0:  # Skip constant features
        correlations.append(0)
        p_values.append(1)
    else:
        corr, pval = pointbiserialr(X_train[:, i], y_train)
        correlations.append(corr)
        p_values.append(pval)


# Create results DataFrame
corr_df = pd.DataFrame({
    "Feature": feature_names,
    "Correlation": correlations,
    "P-value": p_values
})


# Sort by absolute correlation strength
corr_df["Abs_Correlation"] = np.abs(corr_df["Correlation"])
corr_df = corr_df.sort_values("Abs_Correlation", ascending=True)

# Function to analyze zero-value ratio in features
def analyze_zeros(ehr_data, feature_cols, threshold=0.5):

    all_zeros = []
    for feature_name, feature_idx in zip(feature_cols, range(len(feature_cols))):
        zeros_count = np.sum([np.sum(matrix[:, feature_idx] == 0) for matrix in ehr_data["feat_dict"].values()])
        total_values = sum([matrix.shape[0] for matrix in ehr_data["feat_dict"].values()])
        zero_ratio = zeros_count / total_values
        if zero_ratio > threshold:
            all_zeros.append((feature_name, zero_ratio))
    return pd.DataFrame(all_zeros, columns=["Feature", "Zero_Ratio"])

# Analyze zero-value ratio for each feature category
demo_zero_df = analyze_zeros(ehr_data, ehr_data["demo_cols"])
icd_zero_df = analyze_zeros(ehr_data, ehr_data["icd_cols"])
lab_zero_df = analyze_zeros(ehr_data, ehr_data["lab_cols"])
med_zero_df = analyze_zeros(ehr_data, ehr_data["med_cols"])

# Combine all zero-value data
zero_df = pd.concat([demo_zero_df, icd_zero_df, lab_zero_df, med_zero_df])

# Merge zero-value data with std and correlation data
combined_df = pd.merge(pd.merge(std_df, corr_df, on='Feature'), zero_df, on='Feature', how='left')

clinically_relevant = [
    # Labs
    'Creatinine Blood', 'Hemoglobin Blood', 'Hematocrit Blood',
    'Potassium Blood', 'Sodium Blood', 'Glucose Blood',
    'Troponin T Blood', 'Platelet Count Blood', 'Eosinophils Blood',
    'pH Urine', 'pO2 Blood', 'pCO2 Blood', 'Anion Gap Blood',

    # ICD-10
    'I10-I16', 'N17-N19', 'J09-J18', 'E70-E88', 'I30-I52',
    'J40-J47', 'B20-B20',

    # Drugs
    'ANTICOAGULANTS', 'ANTIBIOTICS', 'IMMUNOSUPPRESSANTS',
    'ANTIINFLAM.TUMOR NECROSIS FACTOR INHIBITING AGENTS'
]


thresholds = {
    'zero_ratio': {
        'icd': 0.95,    # ICD codes often sparse
        'med': 0.95,    # Medications often sparse
        'lab': 0.95,    # Labs should rarely be zero
        'demo': 0.9     # Demographics rarely zero
    },
    'std_dev': 0.01,    # Only for continuous features
    'correlation': {
        'min_abs_corr': 0.01,
        'max_pvalue': 0.05
    }
}

# Categorize features
feature_types = {
    'icd': ehr_data["icd_cols"],
    'med': ehr_data["med_cols"],
    'lab': ehr_data["lab_cols"],
    'demo': ehr_data["demo_cols"]
}

# Initialize storage
features_to_remove = []


# Check each feature type separately
for ftype, cols in feature_types.items():
    for feature in cols:
        row = combined_df[combined_df['Feature'] == feature].iloc[0]

        # Skip binary features for variance check
        check_variance = ftype not in ['icd']

        # Apply type-specific rules
        if (row['Zero_Ratio'] > thresholds['zero_ratio'][ftype]) and \
           (not check_variance or row['Std_Dev'] < thresholds['std_dev']) and \
           (np.abs(row['Correlation']) < thresholds['correlation']['min_abs_corr']) and \
           (row['P-value'] > thresholds['correlation']['max_pvalue']):
            features_to_remove.append(feature)




filtered_df = combined_df[~combined_df['Feature'].isin(clinically_relevant)]
# Generate report
removal_df = filtered_df[filtered_df['Feature'].isin(features_to_remove)].sort_values(
    by=['Zero_Ratio', 'Std_Dev', 'Abs_Correlation'],
    ascending=[False, True, True]
)

# Get indices of features to remove
features_to_remove = removal_df['Feature'].tolist()
all_features = ehr_data["feature_cols"]
remove_indices = [i for i, feature in enumerate(all_features)
                 if feature in features_to_remove]

#  Function to remove features
def remove_features(X, remove_indices):
    return np.delete(X, remove_indices, axis=1)

#  Apply to both training and validation sets
X_train_filtered = remove_features(X_train, remove_indices)
X_val_filtered = remove_features(X_val, remove_indices)

# 4. Get remaining feature names
remaining_features = [f for i, f in enumerate(all_features)
                     if i not in remove_indices]

<ipython-input-8-3883940c9813>:60: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  zero_df = pd.concat([demo_zero_df, icd_zero_df, lab_zero_df, med_zero_df])


In [10]:
# print preprocessing result

print(f"\nRecommended features to remove (last) ({len(removal_df)} total):")
print(removal_df[['Feature', 'Zero_Ratio', 'Std_Dev', 'Correlation', 'P-value']])
print("Recommended removal features list:")
print(removal_df['Feature'].tolist())


Recommended features to remove (last) (39 total):
                   Feature  Zero_Ratio   Std_Dev  Correlation   P-value
54                 K65-K68    0.999950  0.015576    -0.008536  0.057676
24                 R40-R46    0.999950  0.102142    -0.003603  0.423045
8                  M80-M85    0.999909  0.007789    -0.004268  0.342633
29                 B25-B34    0.999877  0.029816     0.004573  0.309156
26                 N20-N23    0.999863  0.008993    -0.004928  0.273169
73                 N60-N65    0.999831  0.025824     0.006283  0.162356
47                 G89-G99    0.999813  0.182042     0.007699  0.086893
44                 M50-M54    0.999786  0.000000     0.000000  1.000000
111  Basophils Joint Fluid    0.999758  0.000000     0.000000  1.000000
57                 B65-B83    0.999735  0.006359    -0.003484  0.438443
80                 K40-K46    0.999667  0.000000     0.000000  1.000000
93                 R90-R94    0.999653  0.000000     0.000000  1.000000
39           

In [11]:
SAVE_DIR = "/content"
os.makedirs(SAVE_DIR, exist_ok=True)

train_data = pd.DataFrame(X_train_filtered, columns=remaining_features)
train_data["readmitted_within_30days"] = y_train
train_data.to_csv(os.path.join(SAVE_DIR, "train_processed.csv"), index=False)

val_data = pd.DataFrame(X_val_filtered, columns=remaining_features)
val_data["readmitted_within_30days"] = y_val
val_data.to_csv(os.path.join(SAVE_DIR, "val_processed.csv"), index=False)

In [12]:
def merge_labels(df, ehr_df, is_test=False):
    if is_test:
        return df[["id"]].merge(ehr_df, on="id", how="inner")
    else:
        return df[["id", "readmitted_within_30days"]].merge(ehr_df, on="id", how="inner")

test_ehr = merge_labels(test_df, ehr_df, is_test=True)

def aggregate_ehr_test(df):
    df["ehr_mean"] = df["ehr_matrix"].apply(lambda x: np.mean(x, axis=0))
    X = np.stack(df["ehr_mean"].values)
    return X

X_test = aggregate_ehr_test(test_ehr)
X_test_filtered = remove_features(X_test, remove_indices)

test_data = pd.DataFrame(X_test_filtered, columns=remaining_features)
test_data.to_csv(os.path.join(SAVE_DIR, "test_processed.csv"), index=False)

In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np

last day

In [17]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

rf_model.fit(X_train, y_train)

y_val_pred = rf_model.predict_proba(X_val)[:, 1]

auroc_val = roc_auc_score(y_val, y_val_pred)
print(f"AUROC before preprocessing: {auroc_val}")

AUROC before preprocessing: 0.7583467128124204


In [18]:
X_train_filtered = remove_features(X_train, remove_indices)
X_val_filtered = remove_features(X_val, remove_indices)

rf_model.fit(X_train_filtered, y_train)

y_val_pred = rf_model.predict_proba(X_val_filtered)[:, 1]

auroc_val = roc_auc_score(y_val, y_val_pred)
print(f"AUROC after preprocessing: {auroc_val}")

AUROC after preprocessing: 0.7696181319338177


In [20]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
import numpy as np

random_states = np.random.randint(1, 100, 10)  # 随机生成 10 个不同的 random_state 值

auroc_scores_preprocessed = []
auroc_scores_raw = []

for random_state in random_states:
    rf_model_raw = RandomForestClassifier(n_estimators=100, random_state=random_state, class_weight='balanced')

    rf_model_raw.fit(X_train, y_train)

    y_val_pred_raw = rf_model_raw.predict_proba(X_val)[:, 1]

    auroc_val_raw = roc_auc_score(y_val, y_val_pred_raw)
    auroc_scores_raw.append(auroc_val_raw)

    rf_model_filtered = RandomForestClassifier(n_estimators=100, random_state=random_state, class_weight='balanced')

    rf_model_filtered.fit(X_train_filtered, y_train)

    y_val_pred_filtered = rf_model_filtered.predict_proba(X_val_filtered)[:, 1]

    auroc_val_filtered = roc_auc_score(y_val, y_val_pred_filtered)
    auroc_scores_preprocessed.append(auroc_val_filtered)

average_auroc_raw = np.mean(auroc_scores_raw)
average_auroc_filtered = np.mean(auroc_scores_preprocessed)

print(f"Average AUROC of the model before preprocessing: {average_auroc_raw}")
print(f"Average AUROC of the model after preprocessing: {average_auroc_filtered}")

Average AUROC of the model before preprocessing: 0.757196710222201
Average AUROC of the model after preprocessing: 0.7629648885807192
